ref: https://github.com/NetManAIOps/OmniAnomaly

In [1]:
import os, sys, math

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
os.chdir('/home/maradonam/Implementation/Anomaly/TAD-main/')
print(os.getcwd())

/home/maradonam/Implementation/Anomaly/TAD-main


In [3]:
import scipy
import itertools as it
import string
import re
import random

import tensorflow as tf

import numpy as np
import pandas as pd

from scipy.stats import pearsonr
from scipy import spatial
from tqdm import tqdm 

from tensorflow.keras import Model, layers, optimizers, regularizers, callbacks, activations

from utils.evaluator import evaluate, set_thresholds, compute_traditional_metrics
from utils.evaluator_seg import compute_metrics
from utils.data_loader import _create_sequences
# Univariate
from utils.data_loader import load_kpi, load_IoT_fridge
# Multivariate
from utils.data_loader import load_samsung, load_energy, load_unsw, load_IoT_modbus

# THESE LINES ARE FOR REPRODUCIBILITY
seed = 7
random.seed(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [4]:
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.7
tf_config.allow_soft_placement = True

In [5]:
os.chdir('/home/maradonam/Implementation/Anomaly/TAD-main/baselines/OmniAnomaly/')
print(os.getcwd())

/home/maradonam/Implementation/Anomaly/TAD-main/baselines/OmniAnomaly


In [6]:
import logging
import os
import pickle
import sys
import time
import warnings
from argparse import ArgumentParser
from pprint import pformat, pprint

import numpy as np
import tensorflow as tf
from tfsnippet.examples.utils import MLResults, print_with_title
from tfsnippet.scaffold import VariableSaver
from tfsnippet.utils import get_variables_as_dict, register_config_arguments, Config

from omni_anomaly.eval_methods import pot_eval, bf_search
from omni_anomaly.model import OmniAnomaly
from omni_anomaly.prediction import Predictor
from omni_anomaly.training import Trainer
from omni_anomaly.utils import get_data_dim, get_data, save_z
import main

/home/maradonam/anaconda3/envs/tf1/lib/python3.6/site-packages/tfsnippet/utils/tfver.py:22: DeprecationWarning: Function 'semver.compare' is deprecated. Deprecated since version 2.10.0.  This function will be removed in semver 3. Use the respective 'semver.VersionInfo.compare' instead.
  compare_result = semver.compare(version, tf.__version__)
/home/maradonam/anaconda3/envs/tf1/lib/python3.6/site-packages/tfsnippet/utils/tfver.py:22: DeprecationWarning: Function 'semver.compare' is deprecated. Deprecated since version 2.10.0.  This function will be removed in semver 3. Use the respective 'semver.VersionInfo.compare' instead.
  compare_result = semver.compare(version, tf.__version__)


In [7]:
class ExpConfig(Config):
    # dataset configuration
    dataset = "machine-1-1"
    x_dim = get_data_dim(dataset)

    # model architecture configuration
    use_connected_z_q = True
    use_connected_z_p = True

    # model parameters
    z_dim = 3
    rnn_cell = 'GRU'  # 'GRU', 'LSTM' or 'Basic'
    rnn_num_hidden = 500
    window_length = 36
    stride = 1
    dense_dim = 500
    posterior_flow_type = 'nf'  # 'nf' or None
    nf_layers = 20  # for nf
    max_epoch = 10
    train_start = 0
    max_train_size = None  # `None` means full train set
    batch_size = 50
    l2_reg = 0.0001
    initial_lr = 0.001
    lr_anneal_factor = 0.5
    lr_anneal_epoch_freq = 40
    lr_anneal_step_freq = None
    std_epsilon = 1e-4

    # evaluation parameters
    test_n_z = 1
    test_batch_size = 50
    test_start = 0
    max_test_size = None  # `None` means full test set

    # the range and step-size for score for searching best-f1
    # may vary for different dataset
    bf_search_min = -400.
    bf_search_max = 400.
    bf_search_step_size = 1.

    valid_step_freq = 50 # 100
    gradient_clip_norm = 10.

    early_stop = True  # whether to apply early stop method
    level = 0.01

    # outputs config
    save_z = False  # whether to save sampled z in hidden space
    get_score_on_dim = True  # whether to get score on dim. If `True`, the score will be a 2-dim ndarray
    save_dir = '/home/maradonam/Implementation/Anomaly/TAD-main/baselines/OmniAnomaly/model'
    restore_dir = None  # If not None, restore variables from this dir
    result_dir = '/home/maradonam/Implementation/Anomaly/TAD-main/baselines/OmniAnomaly/result'  # Where to save the result file
    train_score_filename = 'train_score.pkl'
    test_score_filename = 'test_score.pkl'

In [8]:
def OmniAnomaly_seg(data):
    x_train, x_valid, x_test, y_valid, y_test = data

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        anomaly_scores = main(data)
    
    return -anomaly_scores # Inverse of "The larger `reconstruction probability`, the less likely a point is anomaly" to directly indicate the severity of anomaly.

In [9]:
def train_OmniAnomaly(x_train, x_valid, x_test, y_valid, y_test, label_segment, segmentation=False):
    tf.keras.backend.clear_session()

    logging.basicConfig(
        level='INFO',
        format='%(asctime)s [%(levelname)s] %(name)s: %(message)s'
    )

    # construct the model under `variable_scope` named 'model'
    with tf.variable_scope('model') as model_vs:
        model = OmniAnomaly(config=config, name="model")

        # construct the trainer
        trainer = Trainer(model=model,
                          model_vs=model_vs,
                          max_epoch=config.max_epoch,
                          batch_size=config.batch_size,
                          valid_batch_size=config.test_batch_size,
                          initial_lr=config.initial_lr,
                          lr_anneal_epochs=config.lr_anneal_epoch_freq,
                          lr_anneal_factor=config.lr_anneal_factor,
                          grad_clip_norm=config.gradient_clip_norm,
                          valid_step_freq=config.valid_step_freq)

        # construct the predictor
        predictor = Predictor(model, batch_size=config.batch_size, n_z=config.test_n_z, last_point_only=False)

        with tf.Session(config=tf_config).as_default():

            if config.restore_dir is not None:
                # Restore variables from `save_dir`.
                saver = VariableSaver(get_variables_as_dict(model_vs), config.restore_dir)
                saver.restore()

            if config.max_epoch > 0:
                # train the model
                train_start = time.time()
                best_valid_metrics = trainer.fit((x_train, x_valid))
                train_time = (time.time() - train_start) / config.max_epoch
                best_valid_metrics.update({
                    'train_time': train_time
                })
            else:
                best_valid_metrics = {}

            train_score, train_z, train_pred_speed = predictor.get_score(x_train)

            if config.save_z:
                save_z(train_z, 'train_z')

            if x_test is not None:
                # get score of test set
                test_start = time.time()
                test_score, test_z, pred_speed = predictor.get_score(x_test)
                test_time = time.time() - test_start
                if config.save_z:
                    save_z(test_z, 'test_z')
                best_valid_metrics.update({
                    'pred_time': pred_speed,
                    'pred_total_time': test_time
                })
                
                if y_test is not None and len(y_test) >= len(test_score):
                    if config.get_score_on_dim:
                        # get the joint score
                        test_score = -np.sum(test_score, axis=-1)
                        train_score = -np.sum(train_score, axis=-1)
                        
                        if segmentation == False:
                            return compute_traditional_metrics(test_score, _create_sequences(y_test, config.window_length, config.stride))
                        else:
                            return compute_metrics(test_score, _create_sequences(y_test, config.window_length, config.stride), label_segment)

In [10]:
# get config obj
config = ExpConfig()
os.chdir('/home/maradonam/Implementation/Anomaly/TAD-main/')
result_path = './results'

---

# BASE

## Traditional Metric Evaluation

### SAMSUNG

In [11]:
samsung_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [12]:
data = load_samsung(0, 1, decomposition=False, segmentation=False)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    train, valid, test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 36
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)

        samsung_scores['dataset'].append(f'SAMSUNG{i}')
        samsung_scores['avg_f1'].append(np.average(test_scores['f1']))
        samsung_scores['avg_p'].append(np.average(test_scores['precision']))
        samsung_scores['avg_r'].append(np.average(test_scores['recall']))
        samsung_scores['max_f1'].append(np.max(test_scores['f1']))
        samsung_scores['max_p'].append(np.max(test_scores['precision']))
        samsung_scores['max_r'].append(np.max(test_scores['recall']))
        samsung_scores['pr_auc'].append(test_scores['pr_auc'])
        samsung_scores['roc_auc'].append(test_scores['roc_auc'])

  0%|          | 0/3 [00:00<?, ?it/s]

train_values: (21546, 8)
[Epoch 1/10, Step 50] step time: 0.3715s (±1.417s); train time: 11.6s; valid time: 7.02s; loss: 7.90888 (±8.86354); valid loss: 4.99449 (*)
[Epoch 1/10, Step 100] step time: 0.1833s (±0.7131s); train time: 4.089s; valid time: 5.096s; loss: 4.4707 (±0.331983); valid loss: 4.55168 (*)
[Epoch 1/10, Step 150] step time: 0.1665s (±0.6035s); train time: 4.034s; valid time: 4.308s; loss: 4.42919 (±0.338951); valid loss: 4.95905
[Epoch 1/10, Step 200] step time: 0.1652s (±0.6019s); train time: 3.979s; valid time: 4.299s; loss: 4.79471 (±0.480733); valid loss: 5.13743
[Epoch 1/10, Step 250] step time: 0.1653s (±0.5988s); train time: 4.006s; valid time: 4.277s; loss: 4.58324 (±0.539067); valid loss: 5.02127
[Epoch 1/10, Step 300] step time: 0.1667s (±0.6066s); train time: 4.021s; valid time: 4.328s; loss: 4.87803 (±0.6622); valid loss: 5.4367
[Epoch 1/10, Step 350] step time: 0.166s (±0.6033s); train time: 4.006s; valid time: 4.311s; loss: 5.11993 (±0.756584); valid loss

2021-11-27 15:10:30,072 [INFO] tensorflow: Restoring parameters from /tmp/tmpw3rs4_a5/variables.dat-4300


------------------------------ testing ------------------------------


 33%|███▎      | 1/3 [14:28<28:56, 868.09s/it]

...

 67%|██████▋   | 2/3 [28:32<14:20, 860.87s/it]

train_values: (21546, 8)
[Epoch 1/10, Step 50] step time: 0.345s (±1.289s); train time: 10.51s; valid time: 6.781s; loss: 5.46034 (±2.30879); valid loss: 3.98879 (*)
[Epoch 1/10, Step 100] step time: 0.1876s (±0.7424s); train time: 4.103s; valid time: 5.298s; loss: 3.74187 (±0.506751); valid loss: 3.56708 (*)
[Epoch 1/10, Step 150] step time: 0.1693s (±0.6214s); train time: 4.044s; valid time: 4.436s; loss: 3.59333 (±0.42964); valid loss: 3.63907
[Epoch 1/10, Step 200] step time: 0.1671s (±0.6092s); train time: 4.019s; valid time: 4.355s; loss: 3.52016 (±0.509072); valid loss: 3.70019
[Epoch 1/10, Step 250] step time: 0.182s (±0.7076s); train time: 4.06s; valid time: 5.056s; loss: 3.27533 (±0.435183); valid loss: 3.48182 (*)
[Epoch 1/10, Step 300] step time: 0.182s (±0.7059s); train time: 4.076s; valid time: 5.043s; loss: 3.24022 (±0.698059); valid loss: 3.08601 (*)
[Epoch 1/10, Step 350] step time: 0.1655s (±0.5933s); train time: 4.057s; valid time: 4.238s; loss: 2.90124 (±0.363247); 

2021-11-27 15:39:04,381 [INFO] tensorflow: Restoring parameters from /tmp/tmpp08bos81/variables.dat-4100


------------------------------ testing ------------------------------


100%|██████████| 3/3 [43:07<00:00, 865.13s/it]


In [13]:
samsung_results = pd.DataFrame(samsung_scores)
samsung_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/samsung_Omni.csv', index=False)

In [14]:
samsung_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,SAMSUNG0,0.172642,0.700721,0.272685,1.0,1.0,0.322388,0.240327,0.639655
1,SAMSUNG1,0.186795,0.822747,0.224974,1.0,1.0,0.340425,0.256699,0.870899
2,SAMSUNG2,0.056531,0.127873,0.330464,1.0,1.0,0.088996,0.046368,0.533872


### AIOPS KPI

In [15]:
kpi_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [16]:
data = load_kpi(0, 1, decomposition=False, segmentation=True)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    train, valid, test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 60
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)

        kpi_scores['dataset'].append(f'KPI{i}')
        kpi_scores['avg_f1'].append(np.average(test_scores['f1']))
        kpi_scores['avg_p'].append(np.average(test_scores['precision']))
        kpi_scores['avg_r'].append(np.average(test_scores['recall']))
        kpi_scores['max_f1'].append(np.max(test_scores['f1']))
        kpi_scores['max_p'].append(np.max(test_scores['precision']))
        kpi_scores['max_r'].append(np.max(test_scores['recall']))
        kpi_scores['pr_auc'].append(test_scores['pr_auc'])
        kpi_scores['roc_auc'].append(test_scores['roc_auc'])

  0%|          | 0/1 [00:00<?, ?it/s]

train_values: (66822, 1)
[Epoch 1/10, Step 50] step time: 0.7869s (±3.683s); train time: 14.37s; valid time: 25.06s; loss: 5.39268 (±5.59277); valid loss: 3.8755 (*)
[Epoch 1/10, Step 100] step time: 0.5646s (±3.17s); train time: 5.603s; valid time: 22.64s; loss: 3.93623 (±1.13136); valid loss: 3.59133 (*)
[Epoch 1/10, Step 150] step time: 0.5442s (±3.023s); train time: 5.633s; valid time: 21.59s; loss: 7.09562 (±10.3108); valid loss: 4.78568
[Epoch 1/10, Step 200] step time: 0.5461s (±3.044s); train time: 5.575s; valid time: 21.74s; loss: 11.9788 (±35.2854); valid loss: 4.55398
[Epoch 1/10, Step 250] step time: 0.5425s (±3.012s); train time: 5.619s; valid time: 21.52s; loss: 5.55987 (±4.22675); valid loss: 4.34697
[Epoch 1/10, Step 300] step time: 0.5412s (±3.011s); train time: 5.568s; valid time: 21.51s; loss: 6.9618 (±8.81779); valid loss: 5.14725
[Epoch 1/10, Step 350] step time: 0.5471s (±3.043s); train time: 5.634s; valid time: 21.73s; loss: 4.55308 (±1.48681); valid loss: 4.0159

2021-11-27 17:43:45,764 [INFO] tensorflow: Restoring parameters from /tmp/tmpgz29_up2/variables.dat-13200


------------------------------ testing ------------------------------


100%|██████████| 1/1 [2:08:42<00:00, 7722.26s/it]


In [17]:
kpi_results = pd.DataFrame(kpi_scores)
kpi_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/KPI_Omni.csv', index=False)

In [18]:
kpi_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,KPI0,0.089259,0.992591,0.054145,1.0,1.0,0.835864,0.874042,0.971689


### IoT_modbus

In [19]:
modbus_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [20]:
data = load_IoT_modbus(0, 1, decomposition=False, segmentation=True)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    train, valid, test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 36
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)

        modbus_scores['dataset'].append(f'IoT_modbus{i}')
        modbus_scores['avg_f1'].append(np.average(test_scores['f1']))
        modbus_scores['avg_p'].append(np.average(test_scores['precision']))
        modbus_scores['avg_r'].append(np.average(test_scores['recall']))
        modbus_scores['max_f1'].append(np.max(test_scores['f1']))
        modbus_scores['max_p'].append(np.max(test_scores['precision']))
        modbus_scores['max_r'].append(np.max(test_scores['recall']))
        modbus_scores['pr_auc'].append(test_scores['pr_auc'])
        modbus_scores['roc_auc'].append(test_scores['roc_auc'])

  0%|          | 0/1 [00:00<?, ?it/s]

train_values: (15332, 4)
[Epoch 1/10, Step 50] step time: 0.4995s (±2.173s); train time: 10.97s; valid time: 14.05s; loss: 5.65354 (±0.670906); valid loss: 5.59158 (*)
[Epoch 1/10, Step 100] step time: 0.3223s (±1.69s); train time: 4.058s; valid time: 12.07s; loss: 5.26295 (±0.147775); valid loss: 5.17133 (*)
[Epoch 1/10, Step 150] step time: 0.3238s (±1.699s); train time: 4.075s; valid time: 12.13s; loss: 5.17258 (±0.143353); valid loss: 5.12494 (*)
[Epoch 1/10, Step 200] step time: 0.3238s (±1.704s); train time: 4.034s; valid time: 12.17s; loss: 5.0814 (±0.157157); valid loss: 4.95486 (*)
[Epoch 1/10, Step 250] step time: 0.3125s (±1.606s); train time: 4.173s; valid time: 11.47s; loss: 5.13005 (±0.497782); valid loss: 5.01594
[Epoch 1/10, Step 300] step time: 0.3283s (±1.729s); train time: 4.084s; valid time: 12.35s; loss: 4.50386 (±0.283704); valid loss: 4.1322 (*)
train_values: (15332, 4)
[Epoch 2/10, Step 350, ETA 15m 39.62s] step time: 0.3226s (±1.695s); train time: 3.673s; valid

2021-11-27 18:06:22,489 [INFO] tensorflow: Restoring parameters from /tmp/tmp_v0fxvsg/variables.dat-3050


------------------------------ testing ------------------------------


100%|██████████| 1/1 [22:52<00:00, 1372.29s/it]


In [21]:
modbus_results = pd.DataFrame(modbus_scores)
modbus_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/modbus_Omni.csv', index=False)

In [22]:
modbus_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,IoT_modbus0,0.498878,0.665936,0.586784,1.0,1.0,0.784237,0.64286,0.505203


### AIHub Energy

In [23]:
energy_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [24]:
data = load_energy(0, 1, decomposition=False, segmentation=True)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    train, valid, test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 60
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)

        energy_scores['dataset'].append(f'Energy{i}')
        energy_scores['avg_f1'].append(np.average(test_scores['f1']))
        energy_scores['avg_p'].append(np.average(test_scores['precision']))
        energy_scores['avg_r'].append(np.average(test_scores['recall']))
        energy_scores['max_f1'].append(np.max(test_scores['f1']))
        energy_scores['max_p'].append(np.max(test_scores['precision']))
        energy_scores['max_r'].append(np.max(test_scores['recall']))
        energy_scores['pr_auc'].append(test_scores['pr_auc'])
        energy_scores['roc_auc'].append(test_scores['roc_auc'])

  0%|          | 0/7 [00:00<?, ?it/s]

----------------------Combined_LabelledData_112_전압고조파평균.json TRAIN DATA----------------------
----------------------Combined_LabelledData_112_전압고조파평균.json VALIDATION DATA----------------------


 14%|█▍        | 1/7 [12:45<1:16:30, 765.15s/it]

----------------------Combined_LabelledData_127_역률평균.json TRAIN DATA----------------------
----------------------Combined_LabelledData_127_역률평균.json VALIDATION DATA----------------------


 29%|██▊       | 2/7 [22:22<59:04, 708.80s/it]  

...

 86%|████████▌ | 6/7 [2:24:54<25:02, 1502.76s/it]

train_values: (53271, 32)
[Epoch 1/10, Step 50] step time: 0.4299s (±1.564s); train time: 14.92s; valid time: 6.68s; loss: 21.7283 (±58.9281); valid loss: 13.7577 (*)
[Epoch 1/10, Step 100] step time: 0.1845s (±0.4978s); train time: 5.707s; valid time: 3.557s; loss: 0.519756 (±6.57507); valid loss: 17.476
[Epoch 1/10, Step 150] step time: 0.2028s (±0.6213s); train time: 5.738s; valid time: 4.438s; loss: -1.33237 (±3.51969); valid loss: 8.142 (*)
[Epoch 1/10, Step 200] step time: 0.1843s (±0.4963s); train time: 5.706s; valid time: 3.547s; loss: 0.611337 (±9.23589); valid loss: 13.6793
[Epoch 1/10, Step 250] step time: 0.1833s (±0.4896s); train time: 5.709s; valid time: 3.497s; loss: -3.74019 (±3.14821); valid loss: 9.68581
[Epoch 1/10, Step 300] step time: 0.2027s (±0.619s); train time: 5.75s; valid time: 4.426s; loss: -4.66357 (±4.02064); valid loss: 7.60379 (*)
[Epoch 1/10, Step 350] step time: 0.1846s (±0.499s); train time: 5.703s; valid time: 3.564s; loss: -3.05169 (±3.8317); valid 

2021-11-27 22:53:32,801 [INFO] tensorflow: Restoring parameters from /tmp/tmpaegkjpbu/variables.dat-6900


------------------------------ testing ------------------------------


100%|██████████| 7/7 [3:00:33<00:00, 1693.57s/it]


In [25]:
energy_results = pd.DataFrame(energy_scores)
energy_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/Energy_Omni.csv', index=False)

In [26]:
energy_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,Energy0,0.021287,0.017457,0.029164,0.515346,1.0,0.680169,0.339880,0.202352
1,Energy1,0.029317,0.060354,0.031920,0.917874,1.0,0.776164,0.798600,0.716680
2,Energy2,0.040746,0.998640,0.024103,1.000000,1.0,0.993209,0.971649,0.150922
3,Energy3,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-0.000000
4,Energy4,0.098091,0.986296,0.071383,1.000000,1.0,0.893931,0.926963,0.779211
5,Energy5,0.261573,0.998971,0.214362,1.000000,1.0,0.998109,0.997088,0.496916
6,Energy6,0.216635,0.971540,0.181990,1.000000,1.0,0.894768,0.955509,0.937336


---

## Segmentation-based Metric Evaluation

### SAMSUNG

In [11]:
os.chdir('/home/maradonam/Implementation/Anomaly/TAD-main/baselines/OmniAnomaly/')
print(os.getcwd())

/home/maradonam/Implementation/Anomaly/TAD-main/baselines/OmniAnomaly


In [12]:
from main import main

In [13]:
os.chdir('/home/maradonam/Implementation/Anomaly/TAD-main/')
result_path = './results'

In [14]:
samsung_scores = { 'datasets': [], 'max_TaF': [], 'TaP': [], 'TaR': [], 'TaF': [], 'correct_count': [], 'correct_ratio': [], 'thresholds': [] }

In [15]:
data = load_samsung(0, 1, decomposition=False, segmentation=True)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    x_train, x_valid, x_test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    y_segment_valid, y_segment_test = data['y_segment_test'][i][:valid_idx], data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = x_train.shape[1]
        config.window_length = 36
        data=[x_train, x_valid, x_test, y_valid, y_test]
        
        # Inverse of "The larger `reconstruction probability`, the less likely a point is anomaly" to directly indicate the severity of anomaly.
        test_rec = -main(data, config)
        test_scores = compute_metrics(test_rec, y_test, y_segment_test)

        samsung_scores['dataset'].append(f'SAMSUNG{i+1}')
        samsung_scores['max_TaF'].append(max(test_scores['f1']))
        samsung_scores['TaP'].append(test_scores['precision'])
        samsung_scores['TaR'].append(test_scores['recall'])
        samsung_scores['TaF'].append(test_scores['f1'])
        samsung_scores['correct_count'].append(test_scores['count'])
        samsung_scores['correct_ratio'].append(test_scores['ratio'])
        samsung_scores['thresholds'].append(test_scores['thresholds'])

  0%|          | 0/3 [00:00<?, ?it/s]

train_values: (21600, 8)
[Epoch 1/10, Step 50] step time: 0.3625s (±1.339s); train time: 11.36s; valid time: 6.806s; loss: 5.44696 (±1.33937); valid loss: 4.7164 (*)
[Epoch 1/10, Step 100] step time: 0.1791s (±0.6946s); train time: 4.017s; valid time: 4.958s; loss: 4.2902 (±0.576681); valid loss: 4.57302 (*)
[Epoch 1/10, Step 150] step time: 0.1687s (±0.6137s); train time: 4.059s; valid time: 4.394s; loss: 4.06591 (±1.12542); valid loss: 4.95358
[Epoch 1/10, Step 200] step time: 0.1792s (±0.701s); train time: 3.972s; valid time: 5.005s; loss: 4.09644 (±0.552655); valid loss: 4.07865 (*)
[Epoch 1/10, Step 250] step time: 0.1765s (±0.6783s); train time: 4.006s; valid time: 4.838s; loss: 3.84512 (±0.509154); valid loss: 3.86056 (*)
[Epoch 1/10, Step 300] step time: 0.1655s (±0.6009s); train time: 3.993s; valid time: 4.293s; loss: 4.01842 (±0.820712); valid loss: 3.97189
[Epoch 1/10, Step 350] step time: 0.1782s (±0.6859s); train time: 4.026s; valid time: 4.902s; loss: 3.38864 (±0.604752);

2021-12-09 17:40:24,179 [INFO] tensorflow: Restoring parameters from /tmp/tmphqxfsafq/variables.dat-4000


------------------------------ testing ------------------------------
evaluating with unmatch shape: Labels: 9312 vs. Preds: 21565


IndexError: index 9312 is out of bounds for axis 0 with size 9312

In [ ]:
samsung_results = pd.DataFrame(samsung_scores)
samsung_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/samsung_Omni_seg.csv', index=False)

In [ ]:
samsung_results

### AIOPS KPI

In [ ]:
kpi_scores = { 'datasets': [], 'max_TaF': [], 'TaP': [], 'TaR': [], 'TaF': [], 'correct_count': [], 'correct_ratio': [], 'thresholds': [] }

In [ ]:
data = load_kpi(0, 1, decomposition=False, segmentation=True)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    train, valid, test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 60
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=True)

        kpi_scores['dataset'].append('KPI')
        kpi_scores['max_TaF'].append(max(test_scores['f1']))
        kpi_scores['TaP'].append(test_scores['precision'])
        kpi_scores['TaR'].append(test_scores['recall'])
        kpi_scores['TaF'].append(test_scores['f1'])
        kpi_scores['correct_count'].append(test_scores['count'])
        kpi_scores['correct_ratio'].append(test_scores['ratio'])
        kpi_scores['thresholds'].append(test_scores['thresholds'])

In [ ]:
kpi_results = pd.DataFrame(kpi_scores)
kpi_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/KPI_Omni_seg.csv', index=False)

In [ ]:
kpi_results

### IoT_modbus

In [ ]:
modbus_scores = { 'datasets': [], 'max_TaF': [], 'TaP': [], 'TaR': [], 'TaF': [], 'correct_count': [], 'correct_ratio': [], 'thresholds': [] }

In [ ]:
data = load_IoT_modbus(0, 1, decomposition=False, segmentation=True)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    train, valid, test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 36
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=True)

        modbus_scores['dataset'].append('IoT_modbus')
        modbus_scores['max_TaF'].append(max(test_scores['f1']))
        modbus_scores['TaP'].append(test_scores['precision'])
        modbus_scores['TaR'].append(test_scores['recall'])
        modbus_scores['TaF'].append(test_scores['f1'])
        modbus_scores['correct_count'].append(test_scores['count'])
        modbus_scores['correct_ratio'].append(test_scores['ratio'])
        modbus_scores['thresholds'].append(test_scores['thresholds'])

In [ ]:
modbus_results = pd.DataFrame(modbus_scores)
modbus_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/modbus_Omni_seg.csv', index=False)

In [ ]:
modbus_results

### AIHub Energy

In [ ]:
energy_scores = { 'datasets': [], 'max_TaF': [], 'TaP': [], 'TaR': [], 'TaF': [], 'correct_count': [], 'correct_ratio': [], 'thresholds': [] }

In [ ]:
data = load_energy(0, 1, decomposition=False, segmentation=True)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    train, valid, test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 60
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=True)

        energy_scores['dataset'].append('Energy')
        energy_scores['max_TaF'].append(max(test_scores['f1']))
        energy_scores['TaP'].append(test_scores['precision'])
        energy_scores['TaR'].append(test_scores['recall'])
        energy_scores['TaF'].append(test_scores['f1'])
        energy_scores['correct_count'].append(test_scores['count'])
        energy_scores['correct_ratio'].append(test_scores['ratio'])
        energy_scores['thresholds'].append(test_scores['thresholds'])

In [ ]:
energy_results = pd.DataFrame(energy_scores)
energy_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/Energy_Omni_seg.csv', index=False)

In [ ]:
energy_results

---

# Deep Residual-based

## Traditional Metric Evaluation

### SAMSUNG

In [55]:
samsung_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [56]:
save_file_dir = './datasets/samsung/deep_residual'
with open(f'{save_file_dir}/deep_resid.pkl', 'rb') as f:
    data = pickle.load(f)

for i in tqdm(range(len(data['x_train_resid_long']))):
    valid_idx = int(data['x_test_resid_long'][i].shape[0] * 0.3)
    train, valid, test = data['x_train_resid_long'][i], data['x_test_resid_long'][i][:valid_idx], data['x_test_resid_long'][i][valid_idx:]
    y_valid, y_test = data['label_seq'][i][:valid_idx], data['label_seq'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 36
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)

        samsung_scores['dataset'].append(f'SAMSUNG_{i}')
        samsung_scores['avg_f1'].append(np.average(test_scores['f1']))
        samsung_scores['avg_p'].append(np.average(test_scores['precision']))
        samsung_scores['avg_r'].append(np.average(test_scores['recall']))
        samsung_scores['max_f1'].append(np.max(test_scores['f1']))
        samsung_scores['max_p'].append(np.max(test_scores['precision']))
        samsung_scores['max_r'].append(np.max(test_scores['recall']))
        samsung_scores['pr_auc'].append(test_scores['pr_auc'])
        samsung_scores['roc_auc'].append(test_scores['roc_auc'])




  0%|          | 0/3 [00:00<?, ?it/s]/home/maradonam/anaconda3/envs/tf1/lib/python3.6/site-packages/tensorflow/python/util/tf_stack.py:69: ResourceWarning: unclosed file <_io.TextIOWrapper name='temp/P9d04071a-4f8c-11ec-beba-4cd98f01d9de.txt' mode='w' encoding='UTF-8'>
  frame_info))
/home/maradonam/anaconda3/envs/tf1/lib/python3.6/site-packages/tensorflow/python/util/tf_stack.py:69: ResourceWarning: unclosed file <_io.TextIOWrapper name='temp/L9d04071b-4f8c-11ec-beba-4cd98f01d9de.txt' mode='w' encoding='UTF-8'>
  frame_info))


train_values: (21600, 8)
[Epoch 1/10, Step 50] step time: 0.3558s (±1.344s); train time: 10.88s; valid time: 6.955s; loss: 828.997 (±3843.81); valid loss: -0.0349729 (*)
[Epoch 1/10, Step 100] step time: 0.1814s (±0.7063s); train time: 4.04s; valid time: 5.046s; loss: -6.87946 (±2.58676); valid loss: -10.1605 (*)
[Epoch 1/10, Step 150] step time: 0.1671s (±0.6024s); train time: 4.069s; valid time: 4.304s; loss: -10.5429 (±4.08916); valid loss: -10.1166
[Epoch 1/10, Step 200] step time: 0.1819s (±0.7062s); train time: 4.063s; valid time: 5.047s; loss: -13.5759 (±1.75621); valid loss: -13.4406 (*)
[Epoch 1/10, Step 250] step time: 0.1672s (±0.5962s); train time: 4.12s; valid time: 4.262s; loss: -14.8525 (±2.03494); valid loss: -11.7784
[Epoch 1/10, Step 300] step time: 0.1829s (±0.7116s); train time: 4.082s; valid time: 5.08s; loss: -14.3499 (±1.68621); valid loss: -14.5507 (*)
[Epoch 1/10, Step 350] step time: 0.167s (±0.6022s); train time: 4.062s; valid time: 4.303s; loss: -16.2823 (±2

2021-11-28 13:42:06,435 [INFO] tensorflow: Restoring parameters from /tmp/tmpf_41ypl4/variables.dat-4250


------------------------------ testing ------------------------------





 33%|███▎      | 1/3 [14:17<28:35, 857.56s/it]

...




 67%|██████▋   | 2/3 [28:18<14:12, 852.52s/it]

train_values: (21600, 8)
[Epoch 1/10, Step 50] step time: 0.3511s (±1.32s); train time: 10.55s; valid time: 7.043s; loss: 378.901 (±1848.56); valid loss: 4.96941 (*)
[Epoch 1/10, Step 100] step time: 0.1807s (±0.6965s); train time: 4.075s; valid time: 4.976s; loss: -3.40139 (±16.6307); valid loss: -10.3723 (*)
[Epoch 1/10, Step 150] step time: 0.1666s (±0.599s); train time: 4.063s; valid time: 4.284s; loss: -9.47743 (±6.87592); valid loss: -7.54518
[Epoch 1/10, Step 200] step time: 0.1657s (±0.5969s); train time: 4.034s; valid time: 4.264s; loss: -12.9931 (±3.64252); valid loss: -9.58405
[Epoch 1/10, Step 250] step time: 0.1803s (±0.6996s); train time: 4.036s; valid time: 4.995s; loss: -13.9857 (±2.47463); valid loss: -13.0818 (*)
[Epoch 1/10, Step 300] step time: 0.191s (±0.7675s); train time: 4.097s; valid time: 5.469s; loss: -14.3156 (±1.54614); valid loss: -14.3183 (*)
[Epoch 1/10, Step 350] step time: 0.166s (±0.5939s); train time: 4.071s; valid time: 4.245s; loss: -17.2896 (±1.44

2021-11-28 14:10:16,265 [INFO] tensorflow: Restoring parameters from /tmp/tmpl6zqhe85/variables.dat-3800


------------------------------ testing ------------------------------





100%|██████████| 3/3 [42:29<00:00, 852.03s/it]




In [57]:
samsung_results = pd.DataFrame(samsung_scores)
samsung_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/samsung_Omni_CAD.csv', index=False)

In [58]:
samsung_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,SAMSUNG_0,0.205755,0.978788,0.130769,1.0,1.0,0.575928,0.479907,0.703352
1,SAMSUNG_1,0.134922,0.989948,0.083473,1.0,1.0,0.497717,0.501511,0.754869
2,SAMSUNG_2,0.145662,0.932543,0.134662,1.0,1.0,0.406910,0.308851,0.506434


### AIOPS KPI

In [80]:
kpi_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [81]:
save_file_dir = './datasets/KPI/deep_residual'
with open(f'{save_file_dir}/deep_resid.pkl', 'rb') as f:
    data = pickle.load(f)

for i in tqdm(range(len(data['x_train_resid_long']))):
    valid_idx = int(data['x_test_resid_long'][i].shape[0] * 0.3)
    train, valid, test = data['x_train_resid_long'][i], data['x_test_resid_long'][i][:valid_idx], data['x_test_resid_long'][i][valid_idx:]
    y_valid, y_test = data['label_seq'][i][:valid_idx], data['label_seq'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 36
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)

        kpi_scores['dataset'].append('KPI')
        kpi_scores['avg_f1'].append(np.average(test_scores['f1']))
        kpi_scores['avg_p'].append(np.average(test_scores['precision']))
        kpi_scores['avg_r'].append(np.average(test_scores['recall']))
        kpi_scores['max_f1'].append(np.max(test_scores['f1']))
        kpi_scores['max_p'].append(np.max(test_scores['precision']))
        kpi_scores['max_r'].append(np.max(test_scores['recall']))
        kpi_scores['pr_auc'].append(test_scores['pr_auc'])
        kpi_scores['roc_auc'].append(test_scores['roc_auc'])       







  0%|          | 0/1 [00:00<?, ?it/s]

train_values: (66822, 1)
[Epoch 1/10, Step 50] step time: 0.5706s (±2.595s); train time: 11.29s; valid time: 17.33s; loss: 587733 (±4.11388e+06); valid loss: 4.61174 (*)
[Epoch 1/10, Step 100] step time: 0.3807s (±2.099s); train time: 4.055s; valid time: 14.99s; loss: 5.1528 (±3.16153); valid loss: 3.19667 (*)
[Epoch 1/10, Step 150] step time: 0.3696s (±1.993s); train time: 4.255s; valid time: 14.24s; loss: 26.7088 (±72.3266); valid loss: 4.48432
[Epoch 1/10, Step 200] step time: 0.3666s (±2.005s); train time: 4.024s; valid time: 14.32s; loss: 5.13798 (±3.13417); valid loss: 4.64356
[Epoch 1/10, Step 250] step time: 0.3664s (±2.003s); train time: 4.025s; valid time: 14.31s; loss: 24.7776 (±82.0645); valid loss: 44.0268
[Epoch 1/10, Step 300] step time: 0.3727s (±2.044s); train time: 4.047s; valid time: 14.6s; loss: 5.56121 (±5.58552); valid loss: 4.83471
[Epoch 1/10, Step 350] step time: 0.3834s (±2.114s); train time: 4.081s; valid time: 15.11s; loss: 4.72406 (±1.65805); valid loss: 4.

2021-11-28 18:45:53,595 [INFO] tensorflow: Restoring parameters from /tmp/tmp6an4j8pu/variables.dat-13350


------------------------------ testing ------------------------------








100%|██████████| 1/1 [1:31:35<00:00, 5495.82s/it]







In [82]:
kpi_results = pd.DataFrame(kpi_scores)
kpi_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/KPI_Omni_CAD.csv', index=False)

In [83]:
kpi_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,KPI,0.063265,0.975982,0.041753,1.0,1.0,0.352273,0.322352,0.890671


### IoT_modbus

In [84]:
modbus_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [85]:
save_file_dir = './datasets/IoT_modbus/deep_residual'
with open(f'{save_file_dir}/deep_resid.pkl', 'rb') as f:
    data = pickle.load(f)

In [86]:
len(data['x_train_resid_long'])

1

In [87]:
data['x_train_resid_long'][0].shape

(15332, 4)

In [88]:
save_file_dir = './datasets/IoT_modbus/deep_residual'
with open(f'{save_file_dir}/deep_resid.pkl', 'rb') as f:
    data = pickle.load(f)

for i in tqdm(range(len(data['x_train_resid_long']))):
    valid_idx = int(data['x_test_resid_long'][i].shape[0] * 0.3)
    train, valid, test = data['x_train_resid_long'][i], data['x_test_resid_long'][i][:valid_idx], data['x_test_resid_long'][i][valid_idx:]
    y_valid, y_test = data['label_seq'][i][:valid_idx], data['label_seq'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 36
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)
        
        modbus_scores['dataset'].append('IoT_modbus')
        modbus_scores['avg_f1'].append(np.average(test_scores['f1']))
        modbus_scores['avg_p'].append(np.average(test_scores['precision']))
        modbus_scores['avg_r'].append(np.average(test_scores['recall']))
        modbus_scores['max_f1'].append(np.max(test_scores['f1']))
        modbus_scores['max_p'].append(np.max(test_scores['precision']))
        modbus_scores['max_r'].append(np.max(test_scores['recall']))
        modbus_scores['pr_auc'].append(test_scores['pr_auc'])
        modbus_scores['roc_auc'].append(test_scores['roc_auc'])







  0%|          | 0/1 [00:00<?, ?it/s]

train_values: (15332, 4)
[Epoch 1/10, Step 50] step time: 0.5046s (±2.197s); train time: 11.09s; valid time: 14.17s; loss: 80.1817 (±517.537); valid loss: 3.38703 (*)
[Epoch 1/10, Step 100] step time: 0.326s (±1.717s); train time: 4.044s; valid time: 12.27s; loss: 0.0555116 (±2.8589); valid loss: -4.03185 (*)
[Epoch 1/10, Step 150] step time: 0.3101s (±1.6s); train time: 4.094s; valid time: 11.43s; loss: -2.94959 (±1.32642); valid loss: -1.59807
[Epoch 1/10, Step 200] step time: 0.3265s (±1.718s); train time: 4.066s; valid time: 12.27s; loss: -4.01179 (±1.34535); valid loss: -4.77525 (*)
[Epoch 1/10, Step 250] step time: 0.3292s (±1.732s); train time: 4.108s; valid time: 12.37s; loss: -4.30095 (±1.07848); valid loss: -4.87526 (*)
[Epoch 1/10, Step 300] step time: 0.3164s (±1.648s); train time: 4.061s; valid time: 11.78s; loss: -4.60838 (±1.78276); valid loss: -3.3998
train_values: (15332, 4)
[Epoch 2/10, Step 350, ETA 15m 42.77s] step time: 0.3281s (±1.728s); train time: 3.69s; valid t

2021-11-28 19:07:39,623 [INFO] tensorflow: Restoring parameters from /tmp/tmppbpdh8bx/variables.dat-3000


------------------------------ testing ------------------------------








100%|██████████| 1/1 [21:09<00:00, 1269.03s/it]







In [89]:
modbus_results = pd.DataFrame(modbus_scores)
modbus_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/modbus_Omni_CAD.csv', index=False)

In [90]:
modbus_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,IoT_modbus,0.128431,0.52676,0.107109,0.666667,1.0,0.783499,0.548981,0.312778


### AIHub Energy

In [91]:
energy_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [92]:
save_file_dir = './datasets/aihub/deep_residual'
with open(f'{save_file_dir}/deep_resid.pkl', 'rb') as f:
    data = pickle.load(f)

for i in tqdm(range(len(data['x_train_resid_long']))):
    valid_idx = int(data['x_test_resid_long'][i].shape[0] * 0.3)
    train, valid, test = data['x_train_resid_long'][i], data['x_test_resid_long'][i][:valid_idx], data['x_test_resid_long'][i][valid_idx:]
    y_valid, y_test = data['label_seq'][i][:valid_idx], data['label_seq'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 36
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)
        
        energy_scores['dataset'].append('Energy')
        energy_scores['avg_f1'].append(np.average(test_scores['f1']))
        energy_scores['avg_p'].append(np.average(test_scores['precision']))
        energy_scores['avg_r'].append(np.average(test_scores['recall']))
        energy_scores['max_f1'].append(np.max(test_scores['f1']))
        energy_scores['max_p'].append(np.max(test_scores['precision']))
        energy_scores['max_r'].append(np.max(test_scores['recall']))
        energy_scores['pr_auc'].append(test_scores['pr_auc'])
        energy_scores['roc_auc'].append(test_scores['roc_auc'])    







  0%|          | 0/7 [00:00<?, ?it/s]

train_values: (38880, 32)
[Epoch 1/10, Step 50] step time: 0.2863s (±1.031s); train time: 10.44s; valid time: 3.929s; loss: -14.3437 (±34.3165); valid loss: -53.9372 (*)
[Epoch 1/10, Step 100] step time: 0.1232s (±0.2936s); train time: 4.075s; valid time: 2.103s; loss: -52.484 (±11.76); valid loss: -72.2476 (*)
[Epoch 1/10, Step 150] step time: 0.1231s (±0.2925s); train time: 4.077s; valid time: 2.094s; loss: -67.1631 (±5.0869); valid loss: -82.2763 (*)
[Epoch 1/10, Step 200] step time: 0.1102s (±0.1988s); train time: 4.106s; valid time: 1.42s; loss: -70.3314 (±3.86472); valid loss: -76.0224
[Epoch 1/10, Step 250] step time: 0.1096s (±0.1996s); train time: 4.069s; valid time: 1.43s; loss: -68.1841 (±6.59141); valid loss: -80.2361
[Epoch 1/10, Step 300] step time: 0.1234s (±0.2955s); train time: 4.072s; valid time: 2.113s; loss: -73.6972 (±5.5537); valid loss: -85.0068 (*)
[Epoch 1/10, Step 350] step time: 0.1254s (±0.3063s); train time: 4.103s; valid time: 2.183s; loss: -74.7578 (±5.15

2021-11-29 09:06:44,157 [INFO] tensorflow: Restoring parameters from /tmp/tmp17j40skj/variables.dat-5100


------------------------------ testing ------------------------------








 14%|█▍        | 1/7 [15:51<1:35:11, 951.97s/it]

...







 86%|████████▌ | 6/7 [1:43:09<17:54, 1074.04s/it]

train_values: (53271, 32)
[Epoch 1/10, Step 50] step time: 0.312s (±1.13s); train time: 10.45s; valid time: 5.228s; loss: 2.35567 (±67.6057); valid loss: -38.3836 (*)
[Epoch 1/10, Step 100] step time: 0.1445s (±0.4305s); train time: 4.176s; valid time: 3.075s; loss: -47.713 (±6.33246); valid loss: -39.7451 (*)
[Epoch 1/10, Step 150] step time: 0.1295s (±0.3338s); train time: 4.108s; valid time: 2.387s; loss: -54.2783 (±8.24781); valid loss: -18.9661
[Epoch 1/10, Step 200] step time: 0.1454s (±0.4334s); train time: 4.195s; valid time: 3.096s; loss: -61.5998 (±5.79765); valid loss: -52.2895 (*)
[Epoch 1/10, Step 250] step time: 0.1296s (±0.3282s); train time: 4.148s; valid time: 2.349s; loss: -62.0774 (±5.48199); valid loss: -32.0891
[Epoch 1/10, Step 300] step time: 0.1308s (±0.3462s); train time: 4.09s; valid time: 2.471s; loss: -64.5944 (±7.38791); valid loss: -49.7739
[Epoch 1/10, Step 350] step time: 0.1306s (±0.3321s); train time: 4.176s; valid time: 2.371s; loss: -60.1157 (±14.305

2021-11-29 10:58:36,754 [INFO] tensorflow: Restoring parameters from /tmp/tmp9p02wvtt/variables.dat-3800


------------------------------ testing ------------------------------








100%|██████████| 7/7 [2:08:17<00:00, 1204.08s/it]







In [93]:
energy_results = pd.DataFrame(energy_scores)
energy_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/Energy_Omni_CAD.csv', index=False)

In [94]:
energy_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,Energy,0.000640,0.000471,0.000999,0.471061,1.0,0.640437,0.235530,0.383618
1,Energy,0.009140,0.043495,0.006993,0.620690,1.0,0.740147,0.531780,0.475148
2,Energy,0.537862,0.985666,0.489734,1.000000,1.0,0.983160,0.990728,0.769923
3,Energy,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-0.000000
4,Energy,0.225385,0.970039,0.142836,1.000000,1.0,0.898680,0.942957,0.851320
5,Energy,0.101792,0.991711,0.070395,1.000000,1.0,0.992565,0.978473,0.317825
6,Energy,0.260316,0.993959,0.166285,1.000000,1.0,0.743409,0.771548,0.615983


---

## Segmentation-based Metric Evaluation

### SAMSUNG

In [ ]:
samsung_scores = { 'datasets': [], 'max_TaF': [], 'TaP': [], 'TaR': [], 'TaF': [], 'correct_count': [], 'correct_ratio': [], 'thresholds': [] }

In [ ]:
data = load_samsung(0, 1, decomposition=False, segmentation=True)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    train, valid, test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 36
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=True)

        samsung_scores['dataset'].append('SAMSUNG')
        samsung_scores['max_TaF'].append(max(test_scores['f1']))
        samsung_scores['TaP'].append(test_scores['precision'])
        samsung_scores['TaR'].append(test_scores['recall'])
        samsung_scores['TaF'].append(test_scores['f1'])
        samsung_scores['correct_count'].append(test_scores['count'])
        samsung_scores['correct_ratio'].append(test_scores['ratio'])
        samsung_scores['thresholds'].append(test_scores['thresholds'])

In [ ]:
samsung_results = pd.DataFrame(samsung_scores)
samsung_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/samsung_Omni_seg.csv', index=False)

In [ ]:
samsung_results

### AIOPS KPI

In [ ]:
kpi_scores = { 'datasets': [], 'max_TaF': [], 'TaP': [], 'TaR': [], 'TaF': [], 'correct_count': [], 'correct_ratio': [], 'thresholds': [] }

In [ ]:
data = load_kpi(0, 1, decomposition=False, segmentation=True)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    train, valid, test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 60
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=True)

        kpi_scores['dataset'].append('KPI')
        kpi_scores['max_TaF'].append(max(test_scores['f1']))
        kpi_scores['TaP'].append(test_scores['precision'])
        kpi_scores['TaR'].append(test_scores['recall'])
        kpi_scores['TaF'].append(test_scores['f1'])
        kpi_scores['correct_count'].append(test_scores['count'])
        kpi_scores['correct_ratio'].append(test_scores['ratio'])
        kpi_scores['thresholds'].append(test_scores['thresholds'])

In [ ]:
kpi_results = pd.DataFrame(kpi_scores)
kpi_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/KPI_Omni_seg.csv', index=False)

In [ ]:
kpi_results

### IoT_modbus

In [ ]:
modbus_scores = { 'datasets': [], 'max_TaF': [], 'TaP': [], 'TaR': [], 'TaF': [], 'correct_count': [], 'correct_ratio': [], 'thresholds': [] }

In [ ]:
data = load_IoT_modbus(0, 1, decomposition=False, segmentation=True)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    train, valid, test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 36
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=True)

        modbus_scores['dataset'].append('IoT_modbus')
        modbus_scores['max_TaF'].append(max(test_scores['f1']))
        modbus_scores['TaP'].append(test_scores['precision'])
        modbus_scores['TaR'].append(test_scores['recall'])
        modbus_scores['TaF'].append(test_scores['f1'])
        modbus_scores['correct_count'].append(test_scores['count'])
        modbus_scores['correct_ratio'].append(test_scores['ratio'])
        modbus_scores['thresholds'].append(test_scores['thresholds'])

In [ ]:
modbus_results = pd.DataFrame(modbus_scores)
modbus_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/modbus_Omni_seg.csv', index=False)

In [ ]:
modbus_results

### AIHub Energy

In [ ]:
energy_scores = { 'datasets': [], 'max_TaF': [], 'TaP': [], 'TaR': [], 'TaF': [], 'correct_count': [], 'correct_ratio': [], 'thresholds': [] }

In [ ]:
data = load_energy(0, 1, decomposition=False, segmentation=True)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    train, valid, test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 60
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=True)

        energy_scores['dataset'].append('Energy')
        energy_scores['max_TaF'].append(max(test_scores['f1']))
        energy_scores['TaP'].append(test_scores['precision'])
        energy_scores['TaR'].append(test_scores['recall'])
        energy_scores['TaF'].append(test_scores['f1'])
        energy_scores['correct_count'].append(test_scores['count'])
        energy_scores['correct_ratio'].append(test_scores['ratio'])
        energy_scores['thresholds'].append(test_scores['thresholds'])

In [ ]:
energy_results = pd.DataFrame(energy_scores)
energy_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/Energy_Omni_seg.csv', index=False)

In [ ]:
energy_results